In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yelp-reviews-for-sa-finegrained-5-classes-csv/Version_3.2.1.txt
/kaggle/input/yelp-reviews-for-sa-finegrained-5-classes-csv/yelp_review_fine-grained_5_classes_csv/readme.txt
/kaggle/input/yelp-reviews-for-sa-finegrained-5-classes-csv/yelp_review_fine-grained_5_classes_csv/train.csv
/kaggle/input/yelp-reviews-for-sa-finegrained-5-classes-csv/yelp_review_fine-grained_5_classes_csv/test.csv


In [9]:
!pip install torch transformers scikit-learn -q

In [10]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup

In [11]:
train=pd.read_csv('/kaggle/input/yelp-reviews-for-sa-finegrained-5-classes-csv/yelp_review_fine-grained_5_classes_csv/train.csv')
train=train.sample(2000)
train.head()

,class_index,review_text
576349,2,Thought I would give this thrift store another...
116375,1,Disappointed that I was told that my car would...
175972,2,With all the night clubs available in Veags......
615031,5,This place has AMAZING chicken! While my famil...
127080,1,This place Is TERRIBLE. The self parking lot ...


In [12]:
test=pd.read_csv('/kaggle/input/yelp-reviews-for-sa-finegrained-5-classes-csv/yelp_review_fine-grained_5_classes_csv/test.csv')
test=test.sample(1000)
test.head()

,class_index,review_text
38903,4,I don't want to be a party pooper for not givi...
27642,2,Barista can't/won't reheat cold coffee. I was ...
4597,3,Fado seems to be the default hangout for my fr...
34272,5,Busy busy place... great happy hour specials! ...
23673,4,The cookies at COOKIEZ are so thin and yummy.....


In [13]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy

nlp = spacy.load("en_core_web_sm")

nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    words = word_tokenize(text)
    words = [word for word in words if word not in set(stopwords.words('english'))]
    doc = nlp(" ".join(words))
    lemmas = [token.lemma_ for token in doc]
    return " ".join(lemmas)

train['review_text'] = train['review_text'].apply(preprocess_text)
print(train.head())

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
        class_index                                        review_text
576349            2  thought would give thrift store another chance...
116375            1  disappoint tell car would need alternator repl...
175972            2  night club available veag come terrible terrib...
615031            5  place amazing chicken family wait line street ...
127080            1  place terrible self parking lot sooooo small p...


In [14]:
def label_sentiment(class_index):
    if class_index >= 4.0:
        return 2  # positive
    elif class_index <= 2.0:
        return 0  # negative
    else:
        return 1  # neutral

train['sentiment'] = train['class_index'].apply(label_sentiment)
test['sentiment'] = test['class_index'].apply(label_sentiment)


In [15]:
train.isnull().sum()

class_index    0
review_text    0
sentiment      0
dtype: int64

In [16]:
test.isnull().sum()

class_index    0
review_text    0
sentiment      0
dtype: int64

In [46]:
train.head()

AttributeError: 'function' object has no attribute 'head'

In [18]:
test.head()

,class_index,review_text,sentiment
38903,4,I don't want to be a party pooper for not givi...,2
27642,2,Barista can't/won't reheat cold coffee. I was ...,0
4597,3,Fado seems to be the default hangout for my fr...,1
34272,5,Busy busy place... great happy hour specials! ...,2
23673,4,The cookies at COOKIEZ are so thin and yummy.....,2


In [19]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=3,
    output_attentions=False,
    output_hidden_states=False,
)

tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [20]:
def tokenize_pad_truncate(texts, max_len=500):
    encoding = tokenizer(
        texts.tolist(),
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )
    return encoding['input_ids'], encoding['attention_mask']

In [21]:
X_train = train['review_text'].values
y_train = train['sentiment'].values
X_test = test['review_text'].values
y_test = test['sentiment'].values

In [22]:
# Process each split
X_train_ids, X_train_masks = tokenize_pad_truncate(X_train)
X_test_ids, X_test_masks = tokenize_pad_truncate(X_test)

In [23]:
# Convert labels to tensors
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

In [24]:
# Create TensorDataset
train_data = TensorDataset(X_train_ids, X_train_masks, y_train)
test_data = TensorDataset(X_test_ids, X_test_masks, y_test)

In [25]:
# Create DataLoader
train_dataloader = DataLoader(train_data, sampler=RandomSampler(train_data), batch_size=8)
test_dataloader = DataLoader(test_data, sampler=SequentialSampler(test_data), batch_size=8)

In [26]:
import tensorflow as tf

print("--> Checking for physical Tensorflow devices")
for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))

2024-07-07 16:09:26.850997: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 16:09:26.851128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 16:09:26.953035: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


--> Checking for physical Tensorflow devices
: /physical_device:CPU:0
: /physical_device:GPU:0
: /physical_device:GPU:1


In [27]:
# load model and initiate optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8)
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 2e-05
    maximize: False
    weight_decay: 0.01
)

In [32]:
# training function
import torch

def train(model, train_dataloader, optimizer):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        b_input_ids, b_input_mask, b_labels = [b.to(device) for b in batch]
        
        optimizer.zero_grad()
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
    
    avg_loss = total_loss / len(train_dataloader)
    return avg_loss

In [35]:
# validation function
def evaluate(model, val_dataloader):
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    
    with torch.no_grad():
        for batch in val_dataloader:
            b_input_ids, b_input_mask, b_labels = [b.to(device) for b in batch]
            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs.loss
            logits = outputs.logits
            
            total_eval_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            total_eval_accuracy += (preds == b_labels).float().mean().item()
    
    avg_accuracy = total_eval_accuracy / len(val_dataloader)
    avg_loss = total_eval_loss / len(val_dataloader)
    return avg_loss, avg_accuracy

In [36]:
epochs = 5 
# training loop
for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    train_loss = train(model, train_dataloader, optimizer)
    val_loss, val_accuracy = evaluate(model, test_dataloader)
    print(f'Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

Epoch 1/5
Train Loss: 0.7817, Validation Loss: 0.7747, Validation Accuracy: 0.6500
Epoch 2/5
Train Loss: 0.5773, Validation Loss: 0.8314, Validation Accuracy: 0.6200
Epoch 3/5
Train Loss: 0.4212, Validation Loss: 0.6243, Validation Accuracy: 0.7450
Epoch 4/5
Train Loss: 0.2837, Validation Loss: 0.7934, Validation Accuracy: 0.7490
Epoch 5/5
Train Loss: 0.2157, Validation Loss: 0.9868, Validation Accuracy: 0.7150


In [37]:
# Save the model
torch.save(model.state_dict(), 'model.pth')

In [51]:

import torch
from transformers import BertTokenizer, BertForSequenceClassification

def load_model(model_path):
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)  # Adjust num_labels as needed
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model_path = '/kaggle/working/model.pth'
model = load_model(model_path)

def predict_sentiment(text, model, tokenizer):
    # Tokenize
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=1)
    
    sentiment_map = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
    sentiment = sentiment_map[predictions.item()]
    
    return sentiment

def analyze_sentiment(text):
    sentiment = predict_sentiment(text, model, tokenizer)
    print(f"Text: {text}" + "\n")
    print(f"Sentiment: {sentiment}" + "\n" + "\n")


user_review=input("enter the review: "+"\n")
# Example usage
analyze_sentiment(user_review)
analyze_sentiment("The Acolyte is a thrilling addition to the Star Wars universe! Set in the High Republic era, this series offers a fresh perspective on the Force and the Jedi Order. The dark, mysterious tone keeps you on the edge of your seat, while the stellar cast brings complex characters to life. The production value is top-notch, with stunning visuals that transport you to a galaxy far, far away. A must-watch for any Star Wars fan!")
analyze_sentiment("The Acolyte fails to capture the essence of what makes Star Wars great. The plot is convoluted and hard to follow, with too many new characters introduced too quickly. The dark tone feels forced and out of place in the Star Wars universe. The pacing is sluggish, and the dialogue often feels stilted. Hardcore fans might find some enjoyment, but casual viewers will likely be left confused and unimpressed.")
analyze_sentiment("The Acolyte presents an interesting concept with its focus on the dark side during the High Republic era. The series has its strengths, such as high production values and some intriguing character dynamics. However, it also has its weaknesses, including pacing issues and occasionally confusing plot points. While it may not appeal to all Star Wars fans, those interested in exploring different aspects of the universe might find it worth watching.")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


enter the review: 
 Plenty of great individual scenes but not a good enough story to glue them together


Text: Plenty of great individual scenes but not a good enough story to glue them together

Sentiment: Neutral


Text: The Acolyte is a thrilling addition to the Star Wars universe! Set in the High Republic era, this series offers a fresh perspective on the Force and the Jedi Order. The dark, mysterious tone keeps you on the edge of your seat, while the stellar cast brings complex characters to life. The production value is top-notch, with stunning visuals that transport you to a galaxy far, far away. A must-watch for any Star Wars fan!

Sentiment: Positive


Text: The Acolyte fails to capture the essence of what makes Star Wars great. The plot is convoluted and hard to follow, with too many new characters introduced too quickly. The dark tone feels forced and out of place in the Star Wars universe. The pacing is sluggish, and the dialogue often feels stilted. Hardcore fans might find some enjoyment, but casual viewers will likely be left confused and unimpressed.

Sentiment: Negative

